In [ ]:
!pip install imdlib

In [ ]:

import imdlib as imd
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
start_yr = 2010
end_yr = 2020
variable = 'rain'
# Specify the directory where you want to download and store data
download_dir = r"D:\imd_grd" # Replace with your desired p

# Create the directory if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
# Download the data using the 'download_data' function first
imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=download_dir)

# Now, open the downloaded data
data = imd.open_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=download_dir)

In [ ]:
import os
import rasterio
from rasterio.enums import Resampling


# Paths
rainfall_folder = r'D:\tif_rain_2021'  # Folder containing reference rainfall files
ndvi_folder = r'D:\tif_rain_southern_india_2010_2021'  # Folder containing clipped NDVI files
output_folder = r'D:\tif_rain_southern_india_2010_2021_resampled'  # Folder to save resampled files

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process all NDVI files in the folder
for filename in os.listdir(ndvi_folder):
    if filename.endswith('.tif'):
        ndvi_path = os.path.join(ndvi_folder, filename)
        print(f"\nProcessing: {filename}")
        
        with rasterio.open(ndvi_path) as ndvi_src:
            ndvi_data = ndvi_src.read(1)
            print(f"NDVI data shape: {ndvi_data.shape}, dtype: {ndvi_data.dtype}")
            
            # Resample NDVI data using average method
            resampled_ndvi = ndvi_src.read(
                out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
                resampling=Resampling.average
            )
            print(f"Resampled NDVI shape: {resampled_ndvi.shape}, dtype: {resampled_ndvi.dtype}")
            
            # Use the original filename for saving
            output_filename = filename  # Keep the original name (e.g., imd_25_south_2010-01-01.tif)
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the resampled NDVI
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=resampled_ndvi.shape[1],
                width=resampled_ndvi.shape[2],
                count=1,
                dtype=resampled_ndvi.dtype,
                crs=rainfall_crs,
                transform=rainfall_transform,
            ) as dst:
                dst.write(resampled_ndvi[0], 1)
        
        print(f"Resampled NDVI saved to: {output_path}")

print("\nAll NDVI files have been resampled.")



In [ ]:
import os
import glob
import numpy as np
import rasterio
from datetime import datetime
import calendar

def compute_imd_monthly_sum(input_folder, output_folder):
    """
    Compute monthly sum of daily IMD rainfall data.
    
    Parameters:
    input_folder: str, path to folder containing daily IMD GeoTIFF files
    output_folder: str, path to store output monthly summed GeoTIFF files
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get list of all files
    files = glob.glob(os.path.join(input_folder, "imd_25_*.tif"))
    
    # Group files by year and month
    monthly_groups = {}
    for file in files:
        # Extract date from filename
        basename = os.path.basename(file)
        date_str = basename.split('_')[-1].replace('.tif', '')
        date = datetime.strptime(date_str, '%Y-%m-%d')
        
        # Create year-month key
        year_month = f"{date.year}-{date.month:02d}"
        
        if year_month not in monthly_groups:
            monthly_groups[year_month] = []
        monthly_groups[year_month].append(file)
    
    # Process each month
    for year_month, file_list in monthly_groups.items():
        year, month = map(int, year_month.split('-'))
        
        # Read first file to get metadata
        with rasterio.open(file_list[0]) as src:
            meta = src.meta.copy()
            
            # Initialize array to store sum of all days
            monthly_sum = np.zeros((src.height, src.width))
            
            # Process each daily file
            for file in file_list:
                with rasterio.open(file) as daily:
                    data = daily.read(1)
                    
                    # Add daily data to monthly sum (assuming NoData value is -999 or adjust as per your data)
                    valid_mask = data != -999
                    monthly_sum[valid_mask] += data[valid_mask]
            
            # Create output filename
            month_name = calendar.month_name[month]
            output_filename = f"IMD_Rainfall_Monthly_Sum_{year}_{month:02d}_{month_name}.tif"
            output_path = os.path.join(output_folder, output_filename)
            
            # Write output file
            meta.update({"dtype": "float32"})
            with rasterio.open(output_path, 'w', **meta) as dst:
                dst.write(monthly_sum.astype(np.float32), 1)
                
        print(f"Processed {year_month}: {len(file_list)} days")

# Example usage
input_folder = r"D:\tif_rain_2010_2014"
output_folder = r"D:\tif_rain_2010_2014_monthly"

compute_imd_monthly_sum(input_folder, output_folder)
